In [ ]:
import sys 
import pandas as pd
import numpy as np

ratings = pd.read_csv('./BX-Book-Ratings.csv', encoding='latin-1')
items = pd.read_csv('./BX-Books.csv', encoding='latin-1')
users = pd.read_csv('./BX-Users.csv', encoding='latin-1')

In [2]:
rating = ratings[ratings['Book-Rating'] != 0 ]
#rating_0 = ratings[ratings['Book-Rating'] == 0 ]
rating = rating[['User-ID', 'ISBN', 'Book-Rating']].reset_index(drop = True)

In [3]:
#rating은 0점을 제외한 데이터 
rating['Book-Rating'].value_counts()

Book-Rating
8     8335
10    7395
7     6139
9     5732
5     3541
6     2887
4      700
3      460
2      204
1      129
Name: count, dtype: int64

##### items / users 이상치 처리 

In [4]:
## 'Year-Of-Publication' 이상치 == 1378 & 2026
# 1378 = rating 1 개 -> 삭제
# 2026 = rating 1개 아니고 5개라 Book-Author : Kathy Lette가 2002년에 publication 있어서, 2006으로 대체 
items.describe()

items.drop(items[items['Year-Of-Publication'] == 1378].index, inplace = True)
items.loc[items['Year-Of-Publication'] == 2026, 'Year-Of-Publication'] = 2006

## 'Book-Title' 이상치 == name에 "#NAME"
fault_name_idx = items[items['Book-Title'].str.contains("#NAME")].index
items.drop(fault_name_idx, inplace = True)

# ISBN 기준 중복행 제거 
#(ISBN  == '051513628X' 일때 Book title 이 'Key of Light (Key Trilogy (Paperback))',Key of Light' 각각이라 2개 표시 되있는데, 그냥 긴제목으로 남기고 삭제함 )

items.drop_duplicates(inplace = True)
items.drop(items.query("ISBN == '051513628X'").index[0], inplace = True)

# 소문자로 변경
def to_lower(series):
    return series.str.lower()

items['Book-Title'] = to_lower (items['Book-Title'])
items['Book-Author'] = to_lower (items['Book-Author'])
items['Publisher'] = to_lower (items['Publisher'])

# 기본적인 진행 위에까지 했을때 남은 행 44129

In [5]:
#year of publication 전처리 
def categorize_year(year):
    if year < 1800:
        return 'Before 1800'
    elif 1800 <= year < 1900:
        return '1800s'
    elif 1900 <= year < 1950:
        return '1900-1949'
    elif 1950 <= year < 2000:
        return '1950-1999'
    elif 2000 <= year < 2010:
        return '2000-2009'
    elif 2010 <= year < 2020:
        return '2010-2019'
    else:
        return '2020s'

# 'Year-Of-Publication' 열의 데이터 타입을 정수로 변환
items['Year-Of-Publication'] = items['Year-Of-Publication'].astype(int)

# 'Year-Of-Publication' 열 값을 범주화
items['Year-Of-Publication'] = items['Year-Of-Publication'].apply(categorize_year)

In [6]:
import re
items['Book-Title'] = [re.sub(r'[^0-9a-zA-Z:, ]', ' ', str(i)) for i in items['Book-Title']]
items['Main_Title'] = [i.split(' : ')[0] for i in items['Book-Title']]
items['Sub_Title'] = [''.join(i.split(' : ')[1:]) for i in items['Book-Title']]
items['Sub_Title'] = np.where(items['Sub_Title'] == ' ', 'None', items['Sub_Title'])

In [7]:
# NLTK 라이브러리 설치
import nltk
import re 
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# 특수 문자 제거 함수 
def remove_special_characters(text):
    # 정규 표현식을 사용하여 특수 문자 제거
    text = re.sub(r'[^a-zA-Z0-9:\s]', '', text)
    return text

# 어간 추출 함수
def stem_text(text):
    # PorterStemmer 객체 생성
    stemmer = PorterStemmer()
    
    # 텍스트 토큰화
    tokens = word_tokenize(text)
    
    # 각 토큰의 어간 추출 후 다시 조합
    stemmed_text = ' '.join([stemmer.stem(word) for word in tokens])
    return stemmed_text

# 소문자로 변경
def to_lower(series):
    return series.str.lower()

# NaN 값을 빈 문자열로 대체
items['Book-Author'] = items['Book-Author'].fillna('')

# 소문자로 변환 후 어간 추출 함수 적용
#items['Book-Title'] = to_lower(items['Book-Title']).apply(stem_text)
items['Book-Author'] = to_lower(items['Book-Author']).apply(stem_text)
items['Publisher'] = to_lower(items['Publisher']).apply(stem_text)

##### Users

In [8]:
#Age 
#5세 미만, 100세 초과 제거 
del_age_index = users[(users['Age']<5 )|(users['Age']> 100)].index
users.drop(del_age_index, inplace = True)

# 10단위로 범주화하는 함수 정의
bins = range(0, 120, 10)  # 0부터 90까지 10단위로 범주 설정
users['Age_cate'] = pd.cut(users['Age'], bins=bins, labels=[f"{i}-{i+9}" for i in range(0, 110, 10)], right = False)
users['Age_cate'] = users['Age_cate'].apply(lambda x : '5-19' if (x == '0-9')or (x == '10-19') else ('90-100' if x == '90-99' or x == '100-109' else x))

In [9]:
# Location 
# location 시, 주, 도시로 분리 
temp = users["Location"].str.split(", ", expand = True)

In [10]:
# 5개로 분리되는 것들이 총 16개 있어서, 이들은 수작업으로 재분류함 temp[temp[4] == 'usa'] == 2191 번 한개임. 
temp[(temp[3] == 'united kingdom')|(temp[3] == 'usa')|(temp[3] == 'mexico')|(temp[3] == 'spain')|(temp[3] == 'georgia')| (temp[3] == 'canada')]

,0,1,2,3,4
236,coalville,leicestershire,england,united kingdom,None
703,tharston,norwich,england,united kingdom,None
1138,ermington,s. devon,england,united kingdom,None
1150,redmond,or,oregon,usa,None
1258,hauppauge,long island,new york,usa,None
1358,chicago,il,illinois,usa,None
1944,méxico,d.f.,disrito federal,mexico,None
2007,él médano,santa cruz de tenerife,canary islands,spain,None
2189,preston,lancashire,england,united kingdom,None
2191,merrick,acworth,new york,georgia,usa


In [11]:
condition_london = temp[temp[1]== 'london'].index
temp.loc[condition_london, 1] = 'england'

location_error_index= temp[(temp[3] == 'united kingdom')|(temp[3] == 'usa')|(temp[3] == 'mexico')|(temp[3] == 'spain')|(temp[3] == 'georgia')| (temp[3] == 'canada')].index 
temp.loc[location_error_index, 1] = temp.loc[location_error_index, 2]
temp.loc[location_error_index, 2]= temp.loc[location_error_index, 3]
#마지막 열까지 usa 있는것까지 대체 
temp.loc[temp[temp[4] == 'usa'].index, 2] = 'usa' 
temp.columns = ['city','state', 'country', 'error1', 'error2']

In [12]:
users = pd.concat([users, temp[['state', 'country']]], axis = 1)

In [13]:
users[users['country'].isna() == True]

,User-ID,Location,Age,Age_cate,state,country
360,20119,"manchester, new hampshire,",51.0,50-59,"new hampshire,",None
900,49771,"san antonio, texas,",35.0,30-39,"texas,",None
1486,81682,"bluffton, south carolina,",36.0,30-39,"south carolina,",None
1649,90198,"cornwall, new york,",34.0,30-39,"new york,",None
1795,98026,"yarmouth, nova scotia,",40.0,40-49,"nova scotia,",None
2146,117210,"st louis, missouri,",35.0,30-39,"missouri,",None
2789,151861,"sydney, new south wales,",40.0,40-49,"new south wales,",None
3602,195102,"lyon, rhône alpes,",26.0,20-29,"rhône alpes,",None
3770,204359,"brooklyn, new york,",40.0,40-49,"new york,",None


In [14]:
users[users['Location'].str.contains('rhône alpes')]

,User-ID,Location,Age,Age_cate,state,country
3198,172888,"valence, rhône alpes, france",57.0,50-59,rhône alpes,france
3602,195102,"lyon, rhône alpes,",26.0,20-29,"rhône alpes,",None
4596,248854,"lyon, rhône alpes, france",20.0,20-29,rhône alpes,france
4966,268945,"chambéry, rhône alpes, france",18.0,5-19,rhône alpes,france


In [15]:
#country 가 na 인 애들은 해당 state 이 있는 다른 사람들의 Country 찾아서 대체 시킴 
country_na_index = users[users['country'].isna() == True].index
#state에 , 붙어 있는거 다 없애는 전처리 
users.loc[country_na_index,'state'] = users.loc[country_na_index,'state'].str.replace(",", "")
#country 가 na 인 애들은 해당 state 이 있는 다른 사람들의 Country 찾아서 대체 시킴 
users.loc[country_na_index, 'country'] = users.loc[country_na_index, 'state'].apply(lambda x : 'usa' if x in ['new hampshire','texas', 'south carolina', 'new york', 'missouri'] else ('canada' if x == 'nova scotia'
                                                                                                                                              else ('australia' if x == 'new south wales'
                                                                                                                                                    else 'france')))

# state, country 모두 lower case 변경
users['state'] = to_lower(users['state'])
users['country'] = to_lower(users['country'])

In [16]:
#state 전처리
# dc : washington / dc 적절히 구분되어 있어서 그냥 냅둠
# ae : ae 가 Armed Forces Europe 의 약자긴 하다는데 따로 그렇게 쓴게 아닌듯 하여 냅둠
# nm : new mexico 약자인데 하나밖에 없어서 상관없을듯. 그냥 냅둠. 
# state : 'italy' : 대체하기가 애매해서 그냥 내둠 

# state : ga. / ga -> georgia / 'wa' -> wahsington / 'va' -> virginia / 'ca' -> california / 'tx' -> texas / 'co' -> colorado / '-' -> 'brussels'
state_keyword = ['ga.', 'ga', 'wa', 'va', 'ca', 'tx', 'co', '-']
state_kw_index = users[users['state'].isin(state_keyword)].index 

users.loc[state_kw_index, 'state'] = users.loc[state_kw_index, 'state'].apply(lambda x : 'georgia' if x in ['ga.', 'ga'] 
                                         else ('washington' if x == 'wa'
                                                else ('virginia' if x == 'va'
                                                      else ('california' if x== 'ca'
                                                        else ('texas' if x == 'tx'
                                                            else ('colorado' if x == 'co' else 'brussels'))))))

# del : 'my own little world',
users.drop(users[(users['Location'].str.contains('my own little world'))].index, inplace=  True)

# Location 에 'zaragoza' -> state : 'aragon' 
zaragoza_index = users[(users['Location'].str.contains('zaragoza'))].index 
users.loc[zaragoza_index, 'state'] = 'aragon'


In [17]:
# #country 전처리

# del : 'far away...',  'here and there'
users.drop(users[(users['Location'].str.contains('far away...'))].index, inplace= True)
users.drop(users[(users['Location'].str.contains('here and there'))].index, inplace= True)

#country : catalonia, 'catalunya'-> spain 
catalunya_index = users[(users['Location'].str.contains('catalunya'))| (users['Location'].str.contains('catalonia'))].index

users.loc[catalunya_index,'state'] = users.loc[catalunya_index,'state'].apply(lambda x : 'catalunya')
users.loc[catalunya_index,'country'] = users.loc[catalunya_index,'country'].apply(lambda x : 'spain')

#묶음 :  'usa', 'u.s.a.', 'united states', 'united state' / 'phillipines', 'philippines' / 
country_keyword = ['usa', 'u.s.a.', 'united states', 'united state','phillipines', 'philippines' ]
country_kw_index = users[users['country'].isin(country_keyword)].index 

users.loc[country_kw_index, 'country'] = users.loc[country_kw_index, 'country'].apply(lambda x : 'usa' if x in ['usa', 'u.s.a.', 'united states', 'united state'] 
                                         else ('philippines'))


users['country'].unique()

array(['portugal', 'germany', 'usa', 'spain', 'france', 'austria',
       'united kingdom', 'malaysia', 'taiwan', 'canada', 'iran', 'italy',
       'switzerland', 'romania', 'england', 'finland', 'philippines',
       'australia', 'india', 'qatar', 'japan', 'denmark', 'ireland',
       'belgium', 'chile', 'brazil', 'netherlands', 'papua new guinea',
       'new zealand', 'argentina', 'mexico', 'euskal herria', 'sweden',
       'china', 'hong kong', 'hungary', 'singapore', 'poland',
       'south africa', 'orense', 'kuwait', 'dominican republic', 'panama',
       'norway', 'turkey', 'costa rica', 'bulgaria', 'guatemala',
       'madagascar', 'czech republic', 'trinidad and tobago',
       'luxembourg'], dtype=object)

In [18]:
users.drop(columns = ['Location'], inplace = True)

In [19]:
items.drop(columns = ['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], inplace = True)

In [20]:
items.to_csv("./PR_books.csv", index = False)
users.to_csv("./PR_users.csv", index = False)

In [21]:
# user_df와 rating_df를 'User-ID' 기준으로 merge
merged_df = pd.merge(users, rating, on='User-ID')
# merged_df와 item_df를 'ISBN' 기준으로 merge
final_merged_df = pd.merge(merged_df, items, on='ISBN')

In [22]:
final_merged_df.head(5)

,User-ID,Age,Age_cate,state,country,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Main_Title,Sub_Title
0,183,27.0,20-29,porto,portugal,8401422825,9,fahrenheit 451,ray bradburi,1950-1999,"plaza & amp jane editor , s.a .",fahrenheit 451,
1,183,27.0,20-29,porto,portugal,8440630794,7,cuentos del planeta tierra,arthur c. clark,1950-1999,edicion b,cuentos del planeta tierra,
2,80908,30.0,30-39,catalunya,spain,8440630794,7,cuentos del planeta tierra,arthur c. clark,1950-1999,edicion b,cuentos del planeta tierra,
3,183,27.0,20-29,porto,portugal,8476409419,8,estudios sobre el amor,jose ortega y gaset,2000-2009,downtown book center,estudios sobre el amor,
4,242,37.0,30-39,baden-wuerttemberg,germany,3257203659,9,der illustrierte mann erz hlungen,ray bradburi,2000-2009,diogen,der illustrierte mann erz hlungen,


In [23]:
final_merged_df.fillna('0', inplace = True)

In [24]:
final_merged_df.isnull().sum()

User-ID                0
Age                    0
Age_cate               0
state                  0
country                0
ISBN                   0
Book-Rating            0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Main_Title             0
Sub_Title              0
dtype: int64

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split

# KFold 객체 생성
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# 레이블 데이터를 정의
y = final_merged_df['Book-Rating']

# 각 fold에 대해 반복
for i, (train_index, test_index) in enumerate(kf.split(final_merged_df, y)):  # y 인자 추가
    # train과 test 세트 생성
    train_df = final_merged_df.iloc[train_index]
    test_df = final_merged_df.iloc[test_index]

    # 데이터프레임을 CSV 파일로 저장
    train_df.to_csv(f'./models/train_{i}.csv', index=False)
    test_df.to_csv(f'./moels/test_{i}.csv', index=False)
